In [ ]:
import pandas as pd
import pandas_datareader.data as web
import datetime as dt
from sqlalchemy import create_engine, inspect

## Define database connector

In [ ]:
class db_helper:
    def __init__(self, host='localhost', user='root', password='root', db='db'):
        # Sqlalchemy connection
        engine_str = f'mysql+pymysql://{user}:{password}@{host}/{db}'
        self.engine = create_engine(engine_str)
        
    def get_tables(self):
        query = self.engine.execute('SHOW TABLES')
        return [item for tupl in query.fetchall() for item in tupl]
    
    def has_table(self, table):
        return inspect(self.engine).has_table(table)
            
    def get_data_between_dates(self, table, start, end):
        sql = f'''SELECT * FROM {table}
                WHERE Date BETWEEN '{start}' AND '{end}'
                ORDER BY Date'''
        return pd.read_sql(sql, con=self.engine)
    
    def count_rows(self, table, start, end):
        sql = f'''SELECT COUNT(*) FROM {table}
                WHERE Date BETWEEN '{start}' AND '{end}';'''
        return self.engine.execute(sql).fetchall()[0][0]
    
    def df_to_sql(self, df, table, id_):
        # if_exists='append' -> If table exists: Insert new values to the existing table.
        df.to_sql(table.lower(), self.engine, if_exists='append', index=True, index_label=id_)

## Create connection to database

In [ ]:
db = db_helper(db='test_ticker')

## Dowload tickers from Yahoo and store in database

In [ ]:
# Defines the time periods to use
start = dt.datetime(2010, 1, 1)
end = dt.datetime(2021, 1, 1)

# Getting a list af tickers from csv file
tickers = pd.read_csv('Wilshire-5000-Stocks.csv')['Ticker']
tickers = ["AMZN"]
failed_tickers = []

In [ ]:
# Iterate tickers list and download data for all of them in the database
for ticker in tickers:
    try:
        df = web.DataReader(ticker, 'yahoo', start, end)
    except Exception:
        print(f"Ticker '{ticker}' failed to be downloaded")
        failed_tickers.append(ticker)
        continue
    try:
        # If table does not exist it is created automatically
        db.df_to_sql(df, ticker, id_="Date")
    except Exception as e:
        print(f'Table {ticker} already up to date')
        